<a href="https://colab.research.google.com/github/tourihasi/Openstudio/blob/main/scheduel%E3%81%AE%E5%BC%95%E7%B6%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. インポート ------------------------------------------------------------
import openstudio as osd

# 2. ヘルパー関数 ----------------------------------------------------------
# 2.1 Optional返却の ByName で存在確認
def _exists_by_name(optional_getter, name: str) -> bool:
    try:
        opt = optional_getter(name)
        return opt.is_initialized()
    except Exception:
        return False

# 2.2 nameString() をキーに簡易インデックス作成
def _index_by_name(objs):
    d = {}
    for o in objs:
        try:
            d[o.nameString()] = o
        except Exception:
            pass
    return d

# 3. ScheduleTypeLimits の転記（重複回避）----------------------------------
def clone_schedule_type_limits(src_m: osd.model.Model, dst_m: osd.model.Model) -> int:
    dst_idx = _index_by_name(dst_m.getScheduleTypeLimitss())
    copied = 0
    for stl in src_m.getScheduleTypeLimitss():
        name = stl.nameString()
        if name in dst_idx:
            continue
        _ = stl.clone(dst_m)
        copied += 1
        print("[Copy] ScheduleTypeLimits:", name)
    print(f"[Info] ScheduleTypeLimits copied: {copied}")
    return copied

# 4. 内部定義スケジュールの転記（割当てなし／Schedule:Fileは除外）------------
def clone_internal_schedules(src_m: osd.model.Model, dst_m: osd.model.Model) -> None:
    # 4.1 依存する TypeLimits を先に転記
    clone_schedule_type_limits(src_m, dst_m)

    # 4.2 内部定義スケジュール群を対象化
    groups = [
        ("ScheduleRuleset",       src_m.getScheduleRulesets,       dst_m.getScheduleRulesetByName),
        ("ScheduleConstant",      src_m.getScheduleConstants,      dst_m.getScheduleConstantByName),
        ("ScheduleCompact",       src_m.getScheduleCompacts,       dst_m.getScheduleCompactByName),
        ("ScheduleFixedInterval", src_m.getScheduleFixedIntervals, dst_m.getScheduleFixedIntervalByName),
        ("ScheduleYear",          src_m.getScheduleYears,          dst_m.getScheduleYearByName),
        ("ScheduleWeek",          src_m.getScheduleWeeks,          dst_m.getScheduleWeekByName),
        ("ScheduleDay",           src_m.getScheduleDays,           dst_m.getScheduleDayByName),
    ]

    # 4.3 クローン実行（同名はスキップ）
    total = 0
    for label, src_getter, dst_by_name in groups:
        copied = 0
        for sch in src_getter():
            name = sch.nameString()
            if not _exists_by_name(dst_by_name, name):
                _ = sch.clone(dst_m)
                copied += 1
                total += 1
                print(f"[Copy] {label}:", name)
        print(f"[Info] {label} copied: {copied}")

    print(f"[Done] Internal schedule defs copied (excl. TypeLimits): {total}")

# 5. 使い方（例）-----------------------------------------------------------
# src_m = osd.model.Model.load(osd.path("source.osm")).get()
# dst_m = osd.model.Model.load(osd.path("target.osm")).get()
# clone_internal_schedules(src_m, dst_m)
# dst_m.save(osd.path("target_schedules_only.osm"), True)


ModuleNotFoundError: No module named 'openstudio'